In [2]:
import os
import numpy as np
from omegaconf import DictConfig, OmegaConf
import hydra
from hydra import initialize, compose
from hydra.utils import instantiate
from starfish.data import StarfishDataModule
from starfish.model import FasterRCNNLightning

from pprint import pprint
# Set the environment variable HYDRA_FULL_ERROR=1 to see the full stack trace
os.environ['HYDRA_FULL_ERROR'] = '1'

os.environ['PROJECT_ROOT'] = '/Users/moustholmes/starfishDetection'

# Change the current working directory to the project root
os.chdir('/Users/moustholmes/starfishDetection')

initialize(version_base=None, config_path='configs', job_name='notebook')

cfg_train = compose(config_name='main_config',) #overrides=['experiment=effect_gaussian_nll'])

print(OmegaConf.to_yaml(cfg_train))

model:
  _target_: starfish.model.FasterRCNNLightning
  num_classes: 2
  learning_rate: 0.001
  momentum: 0.9
  weight_decay: 0.0005
  optimizer:
    _target_: torch.optim.Adam
    _partial_: true
    lr: 0.001
    weight_decay: 0.01
  scheduler:
    _target_: torch.optim.lr_scheduler.StepLR
    _partial_: true
    step_size: 3
    gamma: 0.1
  compile: false
data:
  _target_: starfish.data.StarfishDataModule
  data_path: data/raw
  batch_size: 4
  train_val_test_split:
  - 0.8
  - 0.1
  - 0.1
  subset: 0.002
  num_workers: 8
trainer:
  _target_: pytorch_lightning.Trainer
  min_epochs: 1
  max_epochs: 2
  accelerator: cpu
  devices: 1



In [3]:
data = instantiate(cfg_train.data)
print(data)
data.setup()
train_dataloader = data.train_dataloader()
batch = next(iter(train_dataloader))
x, y = batch
assert len(x) == data.batch_size
assert len(y) == data.batch_size

Loading 10 images.


In [4]:
model = instantiate(cfg_train.model)
print(model)

FasterRCNNLightning(
  (model): FasterRCNN(
    (transform): GeneralizedRCNNTransform(
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        Resize(min_size=(800,), max_size=1333, mode='bilinear')
    )
    (backbone): BackboneWithFPN(
      (body): IntermediateLayerGetter(
        (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn1): FrozenBatchNorm2d(64, eps=0.0)
        (relu): ReLU(inplace=True)
        (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (layer1): Sequential(
          (0): Bottleneck(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn1): FrozenBatchNorm2d(64, eps=0.0)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): FrozenBatchNorm2d(64, eps=0.0)
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      

In [5]:
trainer = instantiate(cfg_train.trainer)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/moustholmes/miniconda3/envs/starfish/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(
/Users/moustholmes/miniconda3/envs/starfish/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(


In [6]:
trainer.fit(model, data)


  | Name  | Type       | Params
-------------------------------------
0 | model | FasterRCNN | 41.3 M
-------------------------------------
41.1 M    Trainable params
222 K     Non-trainable params
41.3 M    Total params
165.197   Total estimated model params size (MB)


/Users/moustholmes/miniconda3/envs/starfish/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0:  50%|█████     | 1/2 [00:20<00:20, 20.70s/it, v_num=4, train_loss_classifier=0.451, train_loss_box_reg=0.00817, train_loss_objectness=0.337, train_loss_rpn_box_reg=0.0167, train_total_loss=0.813, train_AP=0.0278]

/Users/moustholmes/miniconda3/envs/starfish/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:212: UserWarning: You called `self.log('train_AP', ...)` in your `training_step` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(


Epoch 1: 100%|██████████| 2/2 [00:56<00:00, 28.04s/it, v_num=4, train_loss_classifier=0.130, train_loss_box_reg=0.00915, train_loss_objectness=3.190, train_loss_rpn_box_reg=0.0387, train_total_loss=3.370, train_AP=0.000] 

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|██████████| 2/2 [00:56<00:00, 28.16s/it, v_num=4, train_loss_classifier=0.130, train_loss_box_reg=0.00915, train_loss_objectness=3.190, train_loss_rpn_box_reg=0.0387, train_total_loss=3.370, train_AP=0.000]
